Teste Fadesp - Etapa 01: Carregamento, Limpeza e Análise de Dados de Vendas


In [8]:
# importação do base de dados diretamente do git hub do teste 
import requests
import pandas as pd
url = 'https://raw.githubusercontent.com/fabiopimentel21/desafio-fadesp-processo-seletivo/master/data/dataset_desafio_fadesp.csv'
base_dados_do_teste = pd.read_csv(url, encoding='latin1') # foi encontrado uma dificuldade na leitura do arquivo -> latin-1 ou ISO-8859-1, comum em arquivos em português (por causa dos acentos)

In [9]:
# importação dos metadados para analise da base completa obs: foi indentificado que na base não se encontrava a nomeclatura de toda a base 
metadados_do_teste = pd.read_excel(r'C:\RPA\desafio_projetos\desafio_fadesp\desafio-fadesp-processo-seletivo\metadata.xlsx')

In [10]:
metadados_do_teste.head()

,Coluna,Descrição
0,Row ID,ID do registro
1,Order ID,Identificação do pedido
2,Order Date,Data do pedido
3,Ship Date,Data de envio do pedido
4,Ship Mode,Modo de envio do pedido


In [11]:
# visualização da base para primeiras analises
base_dados_do_teste.head()

,Row ID,Order ID,Order Date,Ship Date,Ship Mode,Customer ID,Customer Name,Segment,City,State,...,Product ID,Category,Sub-Category,Product Name,Sales,Quantity,Discount,Profit,Shipping Cost,Order Priority
0,42433,AG-2011-2040,1/1/2011,6/1/2011,Standard Class,TB-11280,Toby Braunhardt,Consumer,Constantine,Constantine,...,OFF-TEN-10000025,Office Supplies,Storage,"Tenex Lockers, Blue",408.300,2,0.0,106.140,35.46,Medium
1,22253,IN-2011-47883,1/1/2011,8/1/2011,Standard Class,JH-15985,Joseph Holt,Consumer,Wagga Wagga,New South Wales,...,OFF-SU-10000618,Office Supplies,Supplies,"Acme Trimmer, High Speed",120.366,3,0.1,36.036,9.72,Medium
2,48883,HU-2011-1220,1/1/2011,5/1/2011,Second Class,AT-735,Annie Thurman,Consumer,Budapest,Budapest,...,OFF-TEN-10001585,Office Supplies,Storage,"Tenex Box, Single Width",66.120,4,0.0,29.640,8.17,High
3,11731,IT-2011-3647632,1/1/2011,5/1/2011,Second Class,EM-14140,Eugene Moren,Home Office,Stockholm,Stockholm,...,OFF-PA-10001492,Office Supplies,Paper,"Enermax Note Cards, Premium",44.865,3,0.5,-26.055,4.82,High
4,22255,IN-2011-47883,1/1/2011,8/1/2011,Standard Class,JH-15985,Joseph Holt,Consumer,Wagga Wagga,New South Wales,...,FUR-FU-10003447,Furniture,Furnishings,"Eldon Light Bulb, Duo Pack",113.670,5,0.1,37.770,4.70,Medium


In [12]:
# visualização das colunas do df do teste 
base_dados_do_teste.columns

Index(['Row ID', 'Order ID', 'Order Date', 'Ship Date', 'Ship Mode',
       'Customer ID', 'Customer Name', 'Segment', 'City', 'State', 'Country',
       'Postal Code', 'Market', 'Region', 'Product ID', 'Category',
       'Sub-Category', 'Product Name', 'Sales', 'Quantity', 'Discount',
       'Profit', 'Shipping Cost', 'Order Priority'],
      dtype='object')

In [13]:
# tradução das colunas para facilitar analise 
base_dados_do_teste = base_dados_do_teste.rename(columns={
    'Row ID': 'ID da Linha',
    'Order ID': 'ID do Pedido',
    'Order Date': 'Data do Pedido',
    'Ship Date': 'Data de Envio',
    'Ship Mode': 'Modalidade de Envio',
    'Customer ID': 'ID do Cliente',
    'Customer Name': 'Nome do Cliente',
    'Segment': 'Segmento',
    'City': 'Cidade',
    'State': 'Estado',
    'Country': 'País',
    'Postal Code': 'Código Postal',
    'Market': 'Mercado',
    'Region': 'Região',
    'Product ID': 'ID do Produto',
    'Category': 'Categoria',
    'Sub-Category': 'Subcategoria',
    'Product Name': 'Nome do Produto',
    'Sales': 'Vendas',
    'Quantity': 'Quantidade',
    'Discount': 'Desconto',
    'Profit': 'Lucro',
    'Shipping Cost': 'Custo de Envio',
    'Order Priority': 'Prioridade do Pedido'
})

base_dados_do_teste.columns


Index(['ID da Linha', 'ID do Pedido', 'Data do Pedido', 'Data de Envio',
       'Modalidade de Envio', 'ID do Cliente', 'Nome do Cliente', 'Segmento',
       'Cidade', 'Estado', 'País', 'Código Postal', 'Mercado', 'Região',
       'ID do Produto', 'Categoria', 'Subcategoria', 'Nome do Produto',
       'Vendas', 'Quantidade', 'Desconto', 'Lucro', 'Custo de Envio',
       'Prioridade do Pedido'],
      dtype='object')

Iniciando a limpeza dos dados Objetivo: Valores nulos, Dados duplicados e Dados inconsistentes

In [14]:
# ultilizando describe() para ter uma noção inicial do df obs: futuras comparações 
base_dados_do_teste.describe()

,ID da Linha,Código Postal,Vendas,Quantidade,Desconto,Lucro,Custo de Envio
count,51290.00000,9994.000000,51290.000000,51290.000000,51290.000000,51290.000000,51290.000000
mean,25645.50000,55190.379428,246.490581,3.476545,0.142908,28.610982,26.375915
std,14806.29199,32063.693350,487.565361,2.278766,0.212280,174.340972,57.296804
min,1.00000,1040.000000,0.444000,1.000000,0.000000,-6599.978000,0.000000
25%,12823.25000,23223.000000,30.758625,2.000000,0.000000,0.000000,2.610000
50%,25645.50000,56430.500000,85.053000,3.000000,0.000000,9.240000,7.790000
75%,38467.75000,90008.000000,251.053200,5.000000,0.200000,36.810000,24.450000
max,51290.00000,99301.000000,22638.480000,14.000000,0.850000,8399.976000,933.570000


In [15]:
# verificando a qtd de valores nulos em cada coluna
base_dados_do_teste.isnull().sum()

ID da Linha                 0
ID do Pedido                0
Data do Pedido              0
Data de Envio               0
Modalidade de Envio         0
ID do Cliente               0
Nome do Cliente             0
Segmento                    0
Cidade                      0
Estado                      0
País                        0
Código Postal           41296
Mercado                     0
Região                      0
ID do Produto               0
Categoria                   0
Subcategoria                0
Nome do Produto             0
Vendas                      0
Quantidade                  0
Desconto                    0
Lucro                       0
Custo de Envio              0
Prioridade do Pedido        0
dtype: int64